## Service API ##

###  1. Host Service 

#### Systemd Auto run

vi /lib/systemd/system/senti_pos.service

#### Enable Newly Added Service 

sudo systemctl daemon-reload

sudo systemctl enable senti_pos.service

sudo systemctl start senti_pos.service

#### Service Command 

Stop running service 

- systemctl stop senti_pos.service  

Start running service 

- systemctl start senti_pos.service

Restart running service

- systemctl restart senti_pos.service 

Status service

- systemctl status senti_pos.service


### 2. Client 

URL : http://52.77.247.131:5000/api/sentiment/v1.0.0/iuferutlkfu34fs

* Method : POST

* TYPE : JSON

* <font color=green >Example {"text" : "Hello" }</font>






In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*- 
from flask import Flask, jsonify, request
from flask_restful import Resource, Api 
from json import dumps
from polyglot.detect import Detector
from polyglot.text import Text

app = Flask(__name__)
api = Api(app)
app.config['JSON_AS_ASCII'] = False

lexicon = '<--- Lexicon --->'

### -- Lexicon summary -- ###
def dic_score(word):  

    file = open("lexicon/"+lexicon+"/lexicon.txt","r")

    for line in file:

        value = line.split("||")

        if word.encode('utf-8') == value[2].strip() :
            
            try :
            
                return float(value[0])
            
            except :
                
                return 0 
            
def dic_word(sentence):
    
    blob = sentence
    
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0
    
    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative +=1                
                
                if result == 0 :
                    
                    neutral +=1
                
                if result > 0 :
                    
                    positive +=1
         
        sentence = (positive,negative,neutral)
                
    return sentence    

def dic_sentiment(sentence):
    
    blob = sentence
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0 
    
    total = 0

    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative = negative + float(result)                
                
                if result == 0 :
                    
                    neutral = neutral + float(result)
                
                if result > 0 :
                    
                    positive = positive + float(result)                    
                                             
        sentence = positive,negative
                
    return sentence

def dic_word_sum(sentence):
    
    blob = sentence
    text = Text(blob.lower())
    
    positive = 0
    negative =  0
    neutral = 0
    
    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative +=1
                
                if result == 0 :
                    
                    neutral +=1
                
                if result > 0 :
                    
                    positive +=1
        
        sentence = positive+negative+neutral
                
    return sentence

def dic_sentiment_sum(sentence):
    
    blob = sentence
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0 
    total = 0

    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0 
                
                if result < 0 : 
                    
                    negative = negative + float(result) 
                
                if result == 0 :
                    
                    neutral = neutral + float(result) 
                
                if result > 0 :
                    
                    positive = positive + float(result) 
        
        sentence = positive+negative
                
    return sentence

def fraction_sum(word):
    
    try : 
    
        total = (dic_sentiment_sum(word)/dic_word_sum(word))
    
    except:
        
        total = 0
    
    return total


# In[2]:




@app.route('/api/sentiment/v1.0.0/<uuid>', methods=['GET', 'POST'])
def go_sentiment(uuid):
    content = request.json
    # Token id, Call me
    token = "iuferutlkfu34fs" # uuid = token_key
#     onitext = content['text']
    onitext = content['text']
    text = Text(onitext)
    pt = ""
    total = 0 

    # for w in text.words: 
    #    	# text_s = ('%s -->%d'%(w,w.polarity))
    # total+=w.polarity
    count_str = len(text.words)

    for w in text.words:
        total+=w.polarity

    print (total)

    if total > 0:

        point = "positive"

    elif total == 0:

        point = "neutral"

    elif total < 0:

        point = "negative"
     
    if uuid == token:

        sentiment = [
        {
          'score':fraction_sum(onitext),
          'sentiment':point
        }
        ]

        return jsonify({onitext:sentiment})

    else : 

        return jsonify({"id":"NULL"})


if __name__ == '__main__':
    # app.run(host= '0.0.0.0',debug=True)
    app.run(port=5000, threaded=True, host=('0.0.0.0'))

